# API REST
Igual que una interfaz de usuario permite la interacción y comunicación entre un software y una persona, una API (acrónimo de Application Programming Interface) facilita la relación entre dos aplicaciones para el intercambio de mensajes o datos. Un conjunto de funciones y procedimientos que ofrece una biblioteca para que otro software la utilice como capa de abstracción, un espacio de acceso e intercambio de información adicional en la parte superior. Así una se sirve de la información de la otra sin dejar de ser independientes. 

REST es un estilo de arquitectura de software que se utiliza para describir cualquier interfaz entre diferentes sistemas que utilice HTTP para comunicarse. Este término significa REpresentational State Transfer (transferencia de estado representacional), lo que quiere decir que entre dos llamadas cualquiera, el servicio no guarda los datos. Por ejemplo, podemos autenticar a un usuario con su email y contraseña en una llamada, pero la siguiente que hagamos ya se habrá olvidado de la anterior petición de autenticación.

Una API REST es un backend capaz de contestar a las llamadas a una serie de URLs en formato JSON. Es utilizado por muchas empresas gracias a su versatilidad y eficiencia.

La principal ventaja de las API REST es que podemos desarrollar una API en el backend y utilizarla en cualquier dispositivo, ahorrando así mucho tiempo de desarrollo. En el caso de Twitter, la misma API se consume desde Android, iOS y el navegador web.

<img src="https://phpenthusiast.com/theme/assets/images/blog/what_is_rest_api.png" alt="drawing" width="500"/>

## API REST con request
Para utilizar las facilidades de un API REST, es necesario contar con el URI (Uniform Resource Identifier) del recurso y un método de obtener las respuestas ante las peticiones de recursos o acciones sobre el servicio.

![](https://avaldes.com/wp-content/uploads/2017/08/REST_URL_structure.png)

Para manipular los recursos, HTTP nos dota de los siguientes métodos con los cuales debemos operar:

* GET: Para consultar y leer recursos
* POST: Para crear recursos
* PUT: Para editar recursos
* DELETE: Para eliminar recursos.
* PATCH: Para editar partes concretas de un recurso.

Los métodos de uso más común son GET y POST. Por ejemplo, se puede utilizar GET para saber las ultimas actualizaciones de una cuenta de Twitter, o utilizar POST para enviar un Twitter bajo una cuenta de usuario.

In [1]:
import requests
import json

### Public API REST
Existen API REST públicas, es decír de libre consulta para obtener información sobre algunos recursos disponibles de dominio público. Por ejemplo, se puede obtener información libre de la Estación Espacial Internacional (IIS) en el proyecto de fuente abierta [Open Notify](http://open-notify.org/). Aquí podemos obtener algunos datos sobre la Estación Espacial y sus tripulantes.

In [2]:
URL = "http://api.open-notify.org/iss-now.json"
r = requests.get(URL)

In [3]:
r.status_code == requests.codes.ok

True

In [4]:
URL = "http://api.open-notify.org/iss-now.json"
r = requests.get(URL)

if r.status_code == requests.codes.ok:
    data = r.json()
    
    print(json.dumps(data, indent=4))
else:
    print(f"ERROR: Reponse: {r.status_code}")

{
    "message": "success",
    "iss_position": {
        "latitude": "46.8284",
        "longitude": "101.8488"
    },
    "timestamp": 1591101637
}


In [5]:
from datetime import datetime

print(f"Date & Time: {datetime.fromtimestamp(int(data['timestamp']))}")
print(f"Lat: {data['iss_position']['latitude']}, Lon:{data['iss_position']['longitude']}")

Date & Time: 2020-06-02 07:40:37
Lat: 46.8284, Lon:101.8488


### API REST con parametros
La librería `request` tiene como principal utilidad la conformación de un URI a partir de un URL; esto quiere decir, agregarle los parametros requeridos por un GET para obtener la información necesaria.

Por ejemplo, para obtener los momentos en los que se puede tener visibilidad de la Estacion Espacial, se puede hacer una consulta sobre el servicio, agregandole la locación de observación.

In [6]:
# URI: "http://api.open-notify.org/iss-pass.json?lat=LAT&lon=LON"
URL = "http://api.open-notify.org/iss-pass.json"
payload={'lat': '-12.0500',
         'lon': '-77.0500'}
r = requests.get(URL, params=payload)
print(f"GET {r.url}")

if r.status_code == requests.codes.ok:
    data = r.json()
    
    print(json.dumps(data, indent=4))

    # Guardar en un archivo
    with open("temp.json", mode='w') as file:
        json.dump(data, file)
else:
    print(r.status_code)

GET http://api.open-notify.org/iss-pass.json?lat=-12.0500&lon=-77.0500
{
    "message": "success",
    "request": {
        "altitude": 100,
        "datetime": 1591101716,
        "latitude": -12.05,
        "longitude": -77.05,
        "passes": 5
    },
    "response": [
        {
            "duration": 625,
            "risetime": 1591110429
        },
        {
            "duration": 513,
            "risetime": 1591116277
        },
        {
            "duration": 643,
            "risetime": 1591158296
        },
        {
            "duration": 472,
            "risetime": 1591164189
        },
        {
            "duration": 562,
            "risetime": 1591193989
        }
    ]
}


### Ejercicio
Escriba un programa en Python que pida al usuario una coordenada y retorne las próximas 5 vistas posibles de la Estación (fecha-hora inicial/final)    

In [ ]:
import requests
from datetime import datetime

URL = "http://api.open-notify.org/iss-pass.json"
lat, lon = input("Ingrese coordenadas [NS EO]:").split()

payload={'lat': lat,
         'lon': lon}
r = requests.get(URL, params=payload)

if r.status_code == requests.codes.ok:
    data = r.json()
    
    for idx, item in enumerate(data['response'], start=1):
        print(f"{idx}. INI: {datetime.fromtimestamp(int(item['risetime']))} - FIN: {datetime.fromtimestamp(int(item['risetime'] + item['duration']))}")
        
else:
    print("ERROR:", r.status_code)

### Ejercicio
Escriba una aplicación en tkinter que muestre en tiempo real la ubicación en forma de coordenadas de la Estación Espacial Internacional

In [ ]:
import tkinter as tk
import requests
from time import sleep
import threading

class App:
    def __init__(self, master):
        self.master = master
        self.master.geometry("240x100+100+100")
        self.master.resizable(0, 0)
        
        self.URL = "http://api.open-notify.org/iss-now.json"
        
        self.lblNS = tk.Label(self.master, font='Arial 12 bold')
        self.lblWE = tk.Label(self.master, font='Arial 12 bold')
        
        self.lblNS.pack(padx=10, pady=10)
        self.lblWE.pack(padx=10, pady=10)
        
        th = threading.Thread(target=self.query_iis, daemon=True)
        th.start()
        

    def query_iis(self):
        while True:
            r = requests.get(self.URL)

            if r.status_code == requests.codes.ok:
                data = r.json()
                loc_NS = data['iss_position']['latitude']
                loc_WE = data['iss_position']['longitude']

                self.lblNS.config(text=f"{loc_NS} °N")
                self.lblWE.config(text=f"{loc_WE} °W")
            
            sleep(1)

        
root = tk.Tk()
app = App(root)
root.mainloop()

### Public Key API REST
Existen API REST que requieren de un tipo de authenticación por parte del usuario simple, en forma de una código alfanumérico llamado llave (ya sea privada, de consumo, etc.) que debe ser generada como parte de un proceso de creación de cuenta de desarrollador. Por ejemplo, el API REST de [MapQuest](https://developer.mapquest.com/) permite obtener muchos servicios geográficos, pero requiere el uso de una llave de autenticación, por lo que será necesario abrir una cuenta y utilizar la llave para utilizar los recursos.

In [3]:
# Abra una cuenta en MapQuest Developer e insertela aqui
MAPQUEST_KEY = "uIYcZa0GS5VNGH1FyqFme2FykafjI2nH"

El API REST Geocoding permite obtener las coordenadas geográficas a partir de una dirección.

In [4]:
URL = "http://open.mapquestapi.com/geocoding/v1/address"
payload = {'key': MAPQUEST_KEY, 
           'location': 'Av. Primavera 2310, Santiago de Surco, Lima'}
r = requests.get(URL, params=payload)
print(f"GET {r.url}")

if r.status_code == requests.codes.ok:
    data = r.json()
    print(json.dumps(data, indent=4))
    # Guardar en un archivo
    with open("temp.json", mode='w') as file:
        json.dump(data, file)
else:
    print(r.status_code)

GET http://open.mapquestapi.com/geocoding/v1/address?key=uIYcZa0GS5VNGH1FyqFme2FykafjI2nH&location=Av.+Primavera+2310%2C+Santiago+de+Surco%2C+Lima
{
    "info": {
        "statuscode": 0,
        "copyright": {
            "text": "\u00a9 2020 MapQuest, Inc.",
            "imageUrl": "http://api.mqcdn.com/res/mqlogo.gif",
            "imageAltText": "\u00a9 2020 MapQuest, Inc."
        },
        "messages": []
    },
    "options": {
        "maxResults": -1,
        "thumbMaps": true,
        "ignoreLatLngInput": false
    },
    "results": [
        {
            "providedLocation": {
                "location": "Av. Primavera 2310, Santiago de Surco, Lima"
            },
            "locations": [
                {
                    "street": "Avenida Primavera",
                    "adminArea6": "House",
                    "adminArea6Type": "Neighborhood",
                    "adminArea5": "Santiago de Surco",
                    "adminArea5Type": "City",
                    "a

### Ejercicio
Escriba un programa en Python que pida al usuario una direccion y retorne las próximas 5 vistas posibles de la Estación (fecha-hora inicial/final)

    Ingrese la direccion:
 

In [11]:
import requests
from datetime import datetime

URL = "http://open.mapquestapi.com/geocoding/v1/address"
location = input("Ingrese una ubicacion de observacion: ")

payload = {'key': MAPQUEST_KEY, 
           'location': location}
r = requests.get(URL, params=payload)

if r.status_code == requests.codes.ok:
    data = r.json()
    payload={'lat': data['results'][0]['locations'][1]['latLng']['lat'],
             'lon': data['results'][0]['locations'][1]['latLng']['lng']}
    
    URL = "http://api.open-notify.org/iss-pass.json"

    
    r = requests.get(URL, params=payload)

    if r.status_code == requests.codes.ok:
        data = r.json()

        for idx, item in enumerate(data['response'], start=1):
            print(f"{idx}. INI: {datetime.fromtimestamp(int(item['risetime']))} - FIN: {datetime.fromtimestamp(int(item['risetime'] + item['duration']))}")
    else:
        print("ERROR:", r.status_code)
else:
    print("ERROR:", r.status_code)
    
# Av. La Marina 2810, San Miguel, Lima

Ingrese una ubicacion de observacion:  Av. La Marina 2810, San Miguel, Lima


1. INI: 2020-06-02 10:04:16 - FIN: 2020-06-02 10:13:17
2. INI: 2020-06-02 23:31:21 - FIN: 2020-06-02 23:40:28
3. INI: 2020-06-03 01:06:53 - FIN: 2020-06-03 01:17:49
4. INI: 2020-06-03 02:45:35 - FIN: 2020-06-03 02:54:02
5. INI: 2020-06-03 04:25:31 - FIN: 2020-06-03 04:30:53


El API REST Directions permite obtener las direcciones a seguir para llegar desde un punto a otro, utilizando geolocalización.

In [12]:
URL = "http://www.mapquestapi.com/directions/v2/route"
payload = {'key': MAPQUEST_KEY, 
           'from': 'Av. Primavera 2310, Santiago de Surco, Lima', 
           'to': 'La Marina 2810, San Miguel, Lima', 
           'unit': 'k'}  #por defecto devuelve milla, por eso se coloca 'k' para qu edevuelva en km
r = requests.get(URL, params=payload)
print(f"GET {r.url}")

if r.status_code == requests.codes.ok:
    data = r.json()
    
    # Guardar en un archivo
    with open("temp.json", mode='w') as file:
        json.dump(data, file)
else:
    print(r.status_code)

GET http://www.mapquestapi.com/directions/v2/route?key=uIYcZa0GS5VNGH1FyqFme2FykafjI2nH&from=Av.+Primavera+2310%2C+Santiago+de+Surco%2C+Lima&to=La+Marina+2810%2C+San+Miguel%2C+Lima&unit=k


## Ejercicio
Escriba un script que pida dos direcciones (inicio y fin) y muestre un listado informativo de la ruta.

In [13]:
URL = "http://www.mapquestapi.com/directions/v2/route"

_from = input("Ingrese origen: ") 
_to = input("Ingrese destino: ")

payload = {'key': MAPQUEST_KEY, 
           'from': _from, 
           'to': _to, 
           'unit': 'k'}

r = requests.get(URL, params=payload)

if r.status_code == requests.codes.ok:
    data = r.json()
    
    # Muestra la ruta del viaje
    for idx, item in enumerate(data['route']['legs'][0]['maneuvers'], start=1):
        print(f"[{idx}]. {item['narrative']}")
else:
    print("ERROR:", r.status_code)

Ingrese origen:  Av. Primavera 2310, Santiago de Surco, Lima
Ingrese destino:  La Marina 2810, San Miguel, Lima


[1]. Start out going southwest on Avenida Primavera toward Nepeña.
[2]. Turn slight right onto ramp.
[3]. Merge onto PE-1S/Panamericana Sur.
[4]. PE-1S/Panamericana Sur becomes PE-1N.
[5]. Take EXIT 3D.
[6]. Merge onto Javier Prado Este.
[7]. Stay straight to go onto Avenida Javier Prado Oeste.
[8]. Turn slight right onto Faustino Sanchez Carrion.
[9]. Faustino Sanchez Carrion becomes Avenida La Marina.
[10]. AVENIDA LA MARINA.


## Cilentes API a servicios
Existen clientes que pueden ser importados como modulos en Python que se encargan de gestionar todas las llamadas a los diferentes servicios, sin tener que recurrir a llamadas directas. Algunos ejemplos de estos son:

* geopy
* folium

In [16]:
# pip install geopy
from geopy.geocoders import Nominatim  #Nominatim reconoce nombres comerciales

geolocator = Nominatim(user_agent="myapp")
location = geolocator.geocode("Av. Primavera 2310, Santiago de Surco, Lima")

In [17]:
location.address

'Avenida Primavera, Residencial Los Alamos de Monterrico, Monterrico, Santiago de Surco, Lima, 15023, Peru'

In [18]:
location.raw  # devuelve un diccionario con detalles sobre la búsqueda

{'place_id': 94477730,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 34334622,
 'boundingbox': ['-12.1050241', '-12.102967', '-76.9652532', '-76.9626337'],
 'lat': '-12.1038292',
 'lon': '-76.9636151',
 'display_name': 'Avenida Primavera, Residencial Los Alamos de Monterrico, Monterrico, Santiago de Surco, Lima, 15023, Peru',
 'class': 'highway',
 'type': 'primary',
 'importance': 0.71}

In [19]:
# pip install folium
import folium  #modulo que se conecta a un servicio de mapas (liplev?)

In [20]:
m = folium.Map(location=[location.latitude, location.longitude],
               zoom_start=18)   #instancia un objeto mapa

In [21]:
#folium.Marker(location=[location.latitude, location.longitude], popup='UPC Primavera', icon=folium.Icon()).add_to(m)

folium.Marker(location=[location.latitude, location.longitude], popup=location.raw['display_name'], icon=folium.Icon()).add_to(m)

In [22]:
m

In [ ]:
# map de la amistad